In [1]:
import pandas as pd
import numpy as np
import re
import time

import bs4 as bs4
import json

import glob
import tqdm

pd.set_option("max.columns", 131)

#https://strftime.org/
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Resultados do Active Learning

In [4]:
df1 = pd.read_csv("dados/raw_data_with_labels.csv")
df1 = df1[df1['y'].notnull()]
df1.shape

(772, 21)

In [6]:
df2 = pd.read_csv("dados/active_labels1_done.csv")
df2 = df2[df2['y'].notnull()]
df2['novo'] = 1
df2.shape

(200, 24)

In [7]:
df2.head(1)

,Unnamed: 0,link,webpage_url,title,y,query,view_count,like_count,dislike_count,average_rating,subscriber_count,description,categories,tags,upload_date,tempo_desde_pub,channel_url,thumbnail,width,height,resolution,fps,p,novo
0,160,watch?v=_m0UanRL6FE,https://www.youtube.com/watch?v=_m0UanRL6FE,PYTHON ou R para Data Science?,0,data+science,1,147.0,2.0,4.972973,5310.0,Qual a melhor linguagem para,['Science & Technology'],NaN,2020-11-11,6,http://www.youtube.com/channel/UCxaCu5ht7d3jvi...,https://i.ytimg.com/vi_webp/_m0UanRL6FE/maxres...,1920,1080,NaN,24,0.541,1


In [8]:
from sklearn.metrics import roc_auc_score, average_precision_score
average_precision_score(df2['y'],df2['p']), roc_auc_score(df2['y'],df2['p'])

(0.3322777942827373, 0.5974124809741248)

In [9]:
df = pd.concat([df1, df2.drop("p", axis=1)])

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [11]:
df_limpo = pd.DataFrame(index=df.index)
df_limpo['title'] = df['title']
df_limpo['novo'] = df['novo'].fillna(0)

## 1. Limpeza da data

In [12]:
df_limpo['date'] = pd.to_datetime(df['upload_date'], format='%Y-%m-%d')

## 2. Limpeza de Views

In [13]:
df_limpo['views'] = df['view_count'].map(lambda x: 0 if x<0 else int(x))

In [14]:
df_limpo.shape

(972, 4)

## 3. Features

In [18]:
features = pd.DataFrame(index=df_limpo.index)
y = df['y'].copy()

In [19]:
data_extract = datetime.datetime.strptime('2020-11-21', "%Y-%m-%d")

features['tempo_desde_pub'] = (data_extract - df_limpo['date']) / np.timedelta64(1, 'D')
features['views'] = df_limpo['views']
features['views_por_dia'] = features['views'] / features['tempo_desde_pub']
features = features.drop(['tempo_desde_pub'], axis=1)

In [20]:
features.head()

,views,views_por_dia
0,379,94.750000
1,124,20.666667
2,13,1.857143
3,655,72.777778
4,1,0.090909


# Aumenta validação

In [58]:
data_corte = "2020-10-01"
mask_train = (df_limpo['date'] < data_corte) 

mask_val = (df_limpo['date'] >= data_corte) #& (df_limpo['novo'] == 0)

Xtrain, Xval = features[mask_train], features[mask_val]
ytrain, yval = y[mask_train], y[mask_val]
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

((569, 2), (403, 2), (569,), (403,))

In [59]:
Xtrain

,views,views_por_dia
8,76,0.737864
9,17,0.163462
10,211,1.818966
11,6,0.043165
12,14,0.090323
...,...,...
193,7,0.009615
196,105,0.277045
197,3,0.008021
198,232,1.380952


In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

title_train = df_limpo[mask_train]['title']
title_val = df_limpo[mask_val]['title']

title_vec = TfidfVectorizer(min_df=2)
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val)


In [61]:
df_limpo

,title,novo,date,views
0,Hands-on Training in Artificial Intelligence w...,0.0,2020-11-17,379
1,How to LEARN DATA SCIENCE & Machine Learning f...,0.0,2020-11-15,124
2,Data Science Indonesia Profile,0.0,2020-11-14,13
3,Hands-On Training in Artificial Intelligence w...,0.0,2020-11-12,655
4,Hands-On Training in AI with DataScience | Day-5,0.0,2020-11-10,1
...,...,...,...,...
195,Data Science in Africa: Live Q&A with Chris Or...,1.0,2020-11-12,1
196,Machine Learning: Tutorial prático usando apen...,1.0,2019-11-08,105
197,Introdução ao Machine Learning (ML de Zero a 1...,1.0,2019-11-13,3
198,Python free Certification | ibm Free Certifica...,1.0,2020-06-06,232


In [62]:
title_bow_train.shape

(569, 457)

In [63]:
from scipy.sparse import hstack, vstack

Xtrain_wtitle = hstack([Xtrain, title_bow_train])
Xval_wtitle = hstack([Xval, title_bow_val])

In [64]:
Xtrain_wtitle.shape, Xval_wtitle.shape

((569, 459), (403, 459))

In [65]:
mdl = RandomForestClassifier(n_estimators=1000, random_state=0, class_weight="balanced", n_jobs=-1)
mdl.fit(Xtrain_wtitle, ytrain)

RandomForestClassifier(class_weight='balanced', n_estimators=1000, n_jobs=-1,
                       random_state=0)

In [66]:
p = mdl.predict_proba(Xval_wtitle)[:, 1]

In [67]:
from sklearn.metrics import roc_auc_score, average_precision_score

In [68]:
average_precision_score(yval, p), roc_auc_score(yval, p)

(0.5483320689705957, 0.6671111111111112)

dados antigos


ap 0.5235463345585548, auc 0.665264593872563 - mindf=2

aumentando a validação


ap 0.5212442841200238, auc 0.6644575163398694 - mindf=1  
ap 0.5301462501031342, auc 0.6810588235294117 - mindf=2

aumentando o treino mantendo validacao

 
ap 0.5437704978451068 auc 0.6669271869493549 - mindf=2

aumentando tudo

ap 0.5483320689705957 auc 0.6671111111111112 - mindf=2

# Aumenta treino

In [69]:
data_corte = "2020-11-01"
mask_train = (df_limpo['date'] < data_corte)

mask_val = (df_limpo['date'] >= data_corte)

In [71]:
Xtrain, Xval = features[mask_train], features[mask_val]
ytrain, yval = y[mask_train], y[mask_val]
print(Xtrain.shape, Xval.shape, ytrain.shape, yval.shape)

title_train = df_limpo[mask_train]['title']
title_val = df_limpo[mask_val]['title']

title_vec = TfidfVectorizer(min_df=2)
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val)

Xtrain_wtitle = hstack([Xtrain, title_bow_train])
Xval_wtitle = hstack([Xval, title_bow_val])


mdl = RandomForestClassifier(n_estimators=1000, random_state=0, class_weight="balanced", n_jobs=6)
mdl.fit(Xtrain_wtitle, ytrain)

p = mdl.predict_proba(Xval_wtitle)[:, 1]

average_precision_score(yval, p), roc_auc_score(yval, p)

(692, 2) (280, 2) (692,) (280,)


(0.5629330765313236, 0.6903306159420289)

#### mudei a data no mask para 3 mese a frente (10 -> 11), aumentando assim a base de treino

## Fazendo o Active Learning

In [74]:
df_teste = pd.read_csv("dados/raw_data_with_labels.csv")
df_teste = df_teste[df_teste['y'].isnull()]
df_teste.shape

(1217, 21)

In [75]:
df_limpo_teste = pd.DataFrame(index=df_teste.index)
df_limpo_teste['title'] = df_teste['title']
df_limpo_teste['date'] = pd.to_datetime(df_teste['upload_date'], format='%Y-%m-%d')
df_limpo_teste['views'] = df_teste['view_count'].map(lambda x: 0 if x<0 else int(x))
features_teste = pd.DataFrame(index=df_limpo_teste.index)

data_extract = datetime.datetime.strptime('2020-11-21', "%Y-%m-%d")

features_teste['tempo_desde_pub'] = (data_extract - df_limpo_teste['date']) / np.timedelta64(1, 'D')
features_teste['views'] = df_limpo_teste['views']
features_teste['views_por_dia'] = features_teste['views'] / features_teste['tempo_desde_pub']
features_teste = features_teste.drop(['tempo_desde_pub'], axis=1)

In [76]:
data_corte = "2020-11-01"

mask_train = (df_limpo['date'] < data_corte)
mask_val = (df_limpo['date'] >= data_corte)

Xtrain, Xval = pd.concat([features[mask_train], df_limpo[mask_train]['title']],axis=1), pd.concat([features[mask_val], df_limpo[mask_val]['title']],axis=1)
ytrain, yval = y[mask_train], y[mask_val]
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape


((692, 3), (280, 3), (692,), (280,))

In [77]:
Xtrain

,views,views_por_dia,title
6,43,0.977273,Data Science Hands-On Crash Course
7,47,1.068182,Data Science Free Course | Free DataScience Ce...
8,76,0.737864,"O QUE EU FARIA, SE TIVESSE QUE COMEÇAR DATA SC..."
9,17,0.163462,Стоит ли начинать учить data science || Плюсы ...
10,211,1.818966,IIT Roorkee Summer training on Datascience
...,...,...,...
193,7,0.009615,Aprendizado de Máquina - Introdução
196,105,0.277045,Machine Learning: Tutorial prático usando apen...
197,3,0.008021,Introdução ao Machine Learning (ML de Zero a 1...
198,232,1.380952,Python free Certification | ibm Free Certifica...


In [78]:
# Xtrain, Xval = features[mask_train], features[mask_val]
# ytrain, yval = y[mask_train], y[mask_val]
# Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

# title_train = df_limpo[mask_train]['title']
# title_val = df_limpo[mask_val]['title']

title_train = Xtrain['title']
title_val = Xval['title']

title_vec = TfidfVectorizer(min_df=1)
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val)

Xtrain_wtitle = hstack([Xtrain.drop(columns=['title']), title_bow_train])
Xval_wtitle = hstack([Xval.drop(columns=['title']), title_bow_val])


mdl = RandomForestClassifier(n_estimators=1000, random_state=0, class_weight="balanced", n_jobs=6)
mdl.fit(Xtrain_wtitle, ytrain)

p = mdl.predict_proba(Xval_wtitle)[:, 1]

average_precision_score(yval, p), roc_auc_score(yval, p)

(0.5411921885312908, 0.6784137228260869)

In [79]:
roc_auc_score(yval, p)

0.6784137228260869

In [34]:
# from sklearn.pipeline import Pipeline
# from sklearn.compose import ColumnTransformer
# from sklearn.impute import SimpleImputer
# from sklearn.model_selection import ParameterGrid

# PARAM_GRIDS = {
#                 'rf':ParameterGrid({
#                                 'tfidf__transformer__min_df':[1,2,3],
#                                 'model__n_estimators': [ 50, 200,1000],
#                                 #'model__criterion': ['mae', 'mse'], 
#                                 'model__max_depth': [None,  2, 5, 10],
#                                 'model__min_samples_split': [2, 3, 4, 5],
#                                 'model__min_samples_leaf': [1, 2, 3, 4, 5],
#                                 'model__max_features': ['auto', 'sqrt', 'log2'],
#                                 'model__class_weight':["balanced"], 
#                                 'model__n_jobs':[-1]
#                                     }),
#             }

# preprocessor = ColumnTransformer(
#     [
#         ('transformer', TfidfVectorizer(),['title']),
#         #('imputer', SimpleImputer(strategy = 'median'), ['views','views_por_dia'])
#     ])

# model_rf = Pipeline(steps=[('tfidf',preprocessor), ('model',RandomForestClassifier())])

# best_score=np.inf
# best_params=None
# # BUSCANDO HIPERPARAMETROS
# for params in PARAM_GRIDS['rf']:
#     model_rf.set_params(**params)
#     model_rf.fit(Xtrain, ytrain)
#     yval_pred = model_rf.predict_proba(Xval)[:, 1]
#     validation_score = average_precision_score(yval, yval_pred)

#     if validation_score < best_score:
#         best_params = params
#         best_score = validation_score

# print(best_score)
# best_params

In [80]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import ParameterGrid

best_score=-np.inf
best_params=None
best_min_df = 9999
best_ngram_range = 9999
for i in [1,2,3]:
    for j in [(1,1), (1,2), (2,2), (1,3)]:
        print(f"min_df = {i}, ngram_range = {j}")

        title_vec = TfidfVectorizer(min_df=i, ngram_range=j)
        title_bow_train = title_vec.fit_transform(Xtrain['title'])
        title_bow_val = title_vec.transform(Xval['title'])

        Xtrain_loop = hstack([Xtrain.drop(columns=['title']), title_bow_train])
        Xval_loop = hstack([Xval.drop(columns=['title']), title_bow_val])

        PARAM_GRIDS = {
                        'rf':ParameterGrid({
                                        #'tfidf__transformer__min_df':[1,2,3],
                                        'model__n_estimators': [1000],
                                        #'model__criterion': ['mae', 'mse'], 
                                        'model__max_depth': [None,  2, 5, 10],
                                        'model__min_samples_split': [2, 3, 4],
                                        'model__min_samples_leaf': [1, 2, 3],
                                        'model__max_features': ['auto', 'sqrt', 'log2'],
                                        'model__class_weight':["balanced"], 
                                        'model__n_jobs':[6]
                                            }),
                    }

        # preprocessor = ColumnTransformer(
        #     [
        #         ('transformer', TfidfVectorizer(),['title']),
        #         ('imputer', SimpleImputer(strategy = 'median'), ['views','views_por_dia'])
        #     ])

        # model_rf = Pipeline(steps=[('tfidf',preprocessor), ('model',RandomForestClassifier())])
        model_rf = Pipeline(steps=[('model',RandomForestClassifier())])


        # BUSCANDO HIPERPARAMETROS
        for params in PARAM_GRIDS['rf']:
            model_rf.set_params(**params)
            model_rf.fit(Xtrain_loop, ytrain)
            yval_pred = model_rf.predict_proba(Xval_loop)[:, 1]
            validation_score = average_precision_score(yval, yval_pred)
            print(f"melhor = {best_score:.4f}, atual = {validation_score:.4f}")
            if validation_score > best_score:
                best_params = params
                best_score = validation_score
                best_min_df = i
                best_ngram_range = j
        print()

print(best_min_df)
print(best_ngram_range)
print(best_score)
best_params

min_df = 1, ngram_range = (1, 1)
melhor = -inf, atual = 0.5454
melhor = 0.5454, atual = 0.5409
melhor = 0.5454, atual = 0.5429
melhor = 0.5454, atual = 0.5504
melhor = 0.5504, atual = 0.5435
melhor = 0.5504, atual = 0.5523
melhor = 0.5523, atual = 0.5408
melhor = 0.5523, atual = 0.5462
melhor = 0.5523, atual = 0.5354
melhor = 0.5523, atual = 0.5329
melhor = 0.5523, atual = 0.5367
melhor = 0.5523, atual = 0.5373
melhor = 0.5523, atual = 0.5496
melhor = 0.5523, atual = 0.5489
melhor = 0.5523, atual = 0.5517
melhor = 0.5523, atual = 0.5495
melhor = 0.5523, atual = 0.5375
melhor = 0.5523, atual = 0.5372
melhor = 0.5523, atual = 0.5539
melhor = 0.5539, atual = 0.5461
melhor = 0.5539, atual = 0.5584
melhor = 0.5584, atual = 0.5643
melhor = 0.5643, atual = 0.5532
melhor = 0.5643, atual = 0.5667
melhor = 0.5667, atual = 0.5407
melhor = 0.5667, atual = 0.5382
melhor = 0.5667, atual = 0.5616
melhor = 0.5667, atual = 0.5508
melhor = 0.5667, atual = 0.5476
melhor = 0.5667, atual = 0.5476
melhor = 

{'model__class_weight': 'balanced',
 'model__max_depth': 2,
 'model__max_features': 'log2',
 'model__min_samples_leaf': 2,
 'model__min_samples_split': 3,
 'model__n_estimators': 1000,
 'model__n_jobs': 6}

In [81]:
title_vec = TfidfVectorizer(min_df=best_min_df)
title_bow_train = title_vec.fit_transform(Xtrain['title'])
title_bow_val = title_vec.transform(Xval['title'])

Xtrain_loop = hstack([Xtrain.drop(columns='title'), title_bow_train])
Xval_loop = hstack([Xval.drop(columns='title'), title_bow_val])

model_rf.set_params(**best_params)
model_rf.fit(vstack([Xtrain_loop, Xval_loop]), pd.concat([ytrain, yval], axis=0).values)


Pipeline(steps=[('model',
                 RandomForestClassifier(class_weight='balanced', max_depth=2,
                                        max_features='log2', min_samples_leaf=2,
                                        min_samples_split=3, n_estimators=1000,
                                        n_jobs=6))])

In [82]:
Xteste = features_teste
Xteste_loop = title_vec.transform(df_limpo_teste['title'])

Xteste = hstack([Xteste, Xteste_loop])

Xteste

<1217x324 sparse matrix of type '<class 'numpy.float64'>'
	with 9123 stored elements in COOrdinate format>

In [83]:
p = model_rf.predict_proba(Xteste)[:, 1]

y = [1 if x>=0.5 else 0 for x in p ]
df_teste['y'] = y

In [84]:
df_teste

,link,webpage_url,title,y,query,view_count,like_count,dislike_count,average_rating,subscriber_count,description,categories,tags,upload_date,tempo_desde_pub,channel_url,thumbnail,width,height,resolution,fps
156,watch?v=vHaIk07i3zw,https://www.youtube.com/watch?v=vHaIk07i3zw,Improving Global Health Forecasting: Data Scie...,0,data+science,349,5.0,NaN,5.000000,6640.0,Providing patients/end customers with high-qua...,['Nonprofits & Activism'],"['CGD', 'Center for Global Development', 'rese...",2020-11-12,5,http://www.youtube.com/channel/UCWC4mmS_a-0K6J...,https://i.ytimg.com/vi/vHaIk07i3zw/maxresdefau...,1280,720,NaN,60
157,watch?v=GYhD19nmY1w,https://www.youtube.com/watch?v=GYhD19nmY1w,Data Science in Africa: Live Q&A with Chris Or...,0,data+science,1,37.0,2.0,4.794872,6510.0,INSAID is India's first and one of the largest...,['Education'],"['Artificial Intelligence', 'Data Science', 'M...",2020-11-12,5,http://www.youtube.com/channel/UCE0L_4ADPU2iyK...,https://i.ytimg.com/vi/GYhD19nmY1w/maxresdefau...,1280,720,NaN,30
158,watch?v=tgS3NXMfmI0,https://www.youtube.com/watch?v=tgS3NXMfmI0,Hands-On Training in Artificial Intelligence w...,1,data+science,655,32.0,2.0,4.878788,14600.0,NaN,['Entertainment'],NaN,2020-11-12,5,http://www.youtube.com/channel/UCF2_gALht1C1Ns...,https://i.ytimg.com/vi/tgS3NXMfmI0/maxresdefau...,1280,720,NaN,30
159,watch?v=-9FPR90Y1I4,https://www.youtube.com/watch?v=-9FPR90Y1I4,Tableau Data Science Tutorial | Tableau for DS...,0,data+science,2,78.0,3.0,4.851852,2230000.0,Tableau Certification Training: https://www.ed...,['Education'],"['yt:cc=on', 'tableau for data science', 'tabl...",2020-11-11,6,http://www.youtube.com/channel/UCkw4JCwteGrDHI...,https://i.ytimg.com/vi/-9FPR90Y1I4/maxresdefau...,1280,720,NaN,30
160,watch?v=_m0UanRL6FE,https://www.youtube.com/watch?v=_m0UanRL6FE,PYTHON ou R para Data Science?,0,data+science,1,147.0,2.0,4.972973,5310.0,Qual a melhor linguagem para,['Science & Technology'],NaN,2020-11-11,6,http://www.youtube.com/channel/UCxaCu5ht7d3jvi...,https://i.ytimg.com/vi_webp/_m0UanRL6FE/maxres...,1920,1080,NaN,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1984,watch?v=fvF2H85ko9c,https://www.youtube.com/watch?v=fvF2H85ko9c,Getting Started on Kaggle: Writing code to ana...,1,kaggle,35,464.0,12.0,4.899159,68900.0,"In this video,",['Science & Technology'],"['kaggle', 'data science', 'data', 'python', '...",2018-10-16,763,http://www.youtube.com/channel/UCSNeZleDn9c74y...,https://i.ytimg.com/vi_webp/fvF2H85ko9c/maxres...,1280,674,NaN,29
1985,watch?v=ipa7MzNozCY,https://www.youtube.com/watch?v=ipa7MzNozCY,Getting Started on Kaggle: Forking Kernels | K...,0,kaggle,3,37.0,2.0,4.894737,68900.0,"In this video,",['Science & Technology'],"['kaggle', 'data science', 'coding', 'Kaggel',...",2018-10-16,763,http://www.youtube.com/channel/UCSNeZleDn9c74y...,https://i.ytimg.com/vi_webp/ipa7MzNozCY/maxres...,1920,1006,NaN,29
1986,watch?v=LBqmIfAA4A8,https://www.youtube.com/watch?v=LBqmIfAA4A8,Getting Started on Kaggle: Uploading a dataset...,1,kaggle,11,68.0,4.0,4.777778,68900.0,"In this video,",['Science & Technology'],"['kaggle', 'data science', 'coding', 'data', '...",2018-10-16,763,http://www.youtube.com/channel/UCSNeZleDn9c74y...,https://i.ytimg.com/vi_webp/LBqmIfAA4A8/maxres...,1920,958,NaN,29
1987,watch?v=GakrjR4EUds,https://www.youtube.com/watch?v=GakrjR4EUds,Getting Started on Kaggle: R coding in Kernels...,0,kaggle,5,44.0,4.0,4.666666,68900.0,"In this video,",['Science & Technology'],"['Kaggle', 'data science', 'coding', 'rstats',...",2018-10-16,763,http://www.youtube.com/channel/UCSNeZleDn9c74y...,https://i.ytimg.com/vi/GakrjR4EUds/maxresdefau...,1280,672,NaN,29


In [86]:
df_teste.to_csv('dados/raw_data_with_labels_active_learning.csv', index=None)